In [1]:
import torch
import torch.onnx
import torchvision
from torchsummary import summary

In [2]:
# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.
dummy_input = torch.randn(1, 3, 224, 224)
dummy_input_2 = torch.randn(1, 600, 600, 3)
dummy_input_3 = torch.randn(1, 224, 224, 3)

In [3]:
# Obtain your model, it can be also constructed in your script explicitly
# model = torchvision.models.alexnet(pretrained=True)
model = torchvision.models.resnet18(pretrained=True)

In [8]:
class ModelWrapper(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = torchvision.models.resnet18(pretrained=True)   

    def forward(self, x): 
        # x = torch.permute(x, (0, 3, 1, 2))
        x = self.backbone(x)
        return x

In [9]:
model = ModelWrapper()

In [10]:
# model

In [11]:
dummy_output = model(dummy_input)

In [12]:
dummy_output.shape

torch.Size([1, 1000])

In [13]:
#
# Computes the number of trainable parameters
#
# Sourse:
# https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325
#
def get_num_of_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [14]:
get_num_of_params(model)

11689512

In [15]:
def get_size_of_model(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
        
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024**2
    
    print('model size: {:.3f}MB'.format(size_all_mb))

In [16]:
get_size_of_model(model)

model size: 44.629MB


In [17]:
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [18]:
# Invoke export
input_names = ["input_1"]
output_names = ["output_1"]

torch.onnx.export(
    model,
    dummy_input, 
    "models/resnet18chw.onnx",
    opset_version=12,
    input_names=input_names,
    output_names=output_names
)

# ONNX to ORT

In [27]:
MODEL_NAME="./models/resnet18.onnx"

In [28]:
!python3 -m onnxruntime.tools.convert_onnx_models_to_ort {MODEL_NAME}

Converting models with optimization style 'Fixed' and level 'all'
Converting optimized ONNX model /Users/19839701/Developer/python/notebooks_test/models/resnet18.onnx to ORT format model /Users/19839701/Developer/python/notebooks_test/models/resnet18.ort
Converted 1/1 models successfully.
Generating config file from ORT format models with optimization style 'Fixed' and level 'all'
2022-10-21 11:21:41,065 ort_format_model.utils [INFO] - Created config in /Users/19839701/Developer/python/notebooks_test/models/resnet18.required_operators.config
Converting models with optimization style 'Runtime' and level 'all'
Converting optimized ONNX model /Users/19839701/Developer/python/notebooks_test/models/resnet18.onnx to ORT format model /Users/19839701/Developer/python/notebooks_test/models/resnet18.with_runtime_opt.ort
Converted 1/1 models successfully.
Converting models again without runtime optimizations to generate a complete config file. These converted models are temporary and will be dele

In [ ]:
python -m onnxruntime.tools.convert_onnx_models_to_ort ./models/idp/resnet18_mobile_numbers_quantized/resnet18_mobile_numbers_quantized.onnx

# Inspecting model

In [2]:
import onnx

In [3]:
!ls models

alexnet.onnx               en_PP-OCRv3_rec_slim_infer
capturing_2305.onnx        resnet18.mlmodel
dbnet.onnx                 resnet18.onnx
en_PP-OCRv3_rec_infer      resnet18chw.onnx


In [4]:
# Load the ONNX model
onnx_model = onnx.load("models/capturing_2305.onnx")

# Check that the IR is well formed
onnx.checker.check_model(onnx_model)

In [29]:
# Print a human readable representation of the graph
# print(onnx.helper.printable_graph(onnx_model.graph))

In [18]:
!pwd

/Users/19839701/Developer/python/notebooks


In [20]:
!cd ../capturing/ && pwd

/Users/19839701/Developer/python/capturing


In [21]:
!cd ../capturing/ && python inference.py

Traceback (most recent call last):
  File "inference.py", line 12, in <module>
    model = onnxruntime.InferenceSession(path2_best_model,
  File "/Users/19839701/Developer/python/p3env/lib/python3.8/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py", line 335, in __init__
    self._create_inference_session(providers, provider_options, disabled_optimizers)
  File "/Users/19839701/Developer/python/p3env/lib/python3.8/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py", line 370, in _create_inference_session
    sess = C.InferenceSession(session_options, self._model_path, True, self._read_config_from_model)
onnxruntime.capi.onnxruntime_pybind11_state.NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from ./capturing2405.onnx failed:Load model ./capturing2405.onnx failed. File doesn't exist


# pytorch to coreml
https://coremltools.readme.io/docs/pytorch-conversion

In [14]:
# !pip install coremltools

In [15]:
import coremltools as ct

In [6]:
model = model.eval()

In [8]:
traced_model = torch.jit.trace(model, dummy_input)
out = traced_model(dummy_input)

In [10]:
out.shape

torch.Size([1, 1000])

In [16]:
# traced_model

In [18]:
# Using image_input in the inputs parameter:
# Convert to Core ML using the Unified Conversion API.
coreml_model = ct.convert(
    traced_model,
#     inputs=[ct.TensorType(shape=dummy_input.shape)]
    inputs=[ct.ImageType(shape=dummy_input.shape)]
)

Running MIL Common passes:   0%|                                                               | 0/34 [00:00<?, ? passes/s]/Users/19839701/Developer/python/p3env/lib/python3.8/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:101: UserWarning: Input, 'x.1', of the source model, has been renamed to 'x_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/Users/19839701/Developer/python/p3env/lib/python3.8/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '362', of the source model, has been renamed to 'var_362' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████████████████████████████████████| 184/184 [00:00<00:00, 209.11 ops/s]


In [21]:
# Save the converted model.
coreml_model.save("resnet18.mlmodel")